In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
df = pd.read_csv("essays.csv",sep=",", encoding='cp1252')
df.head()



,#AUTHID,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,1997_504851.txt,"Well, right now I just woke up from a mid-day ...",n,y,y,n,y
1,1997_605191.txt,"Well, here we go with the stream of consciousn...",n,n,y,n,n
2,1997_687252.txt,An open keyboard and buttons to push. The thin...,n,y,n,y,y
3,1997_568848.txt,I can't believe it! It's really happening! M...,y,n,y,y,n
4,1997_688160.txt,"Well, here I go with the good old stream of co...",y,n,y,n,y


BERT Classification


In [3]:

import preprocessor as p
df['TEXT'] = df.apply(lambda row : p.clean(str(row['TEXT'])), axis = 1)
df['cEXT'] = df['cEXT'].map(
                   {'y':1 ,'n':0})
df.head()

,#AUTHID,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,1997_504851.txt,"Well, right now I just woke up from a mid-day ...",0,y,y,n,y
1,1997_605191.txt,"Well, here we go with the stream of consciousn...",0,n,y,n,n
2,1997_687252.txt,An open keyboard and buttons to push. The thin...,0,y,n,y,y
3,1997_568848.txt,I can't believe it! It's really happening! My ...,1,n,y,y,n
4,1997_688160.txt,"Well, here I go with the good old stream of co...",1,n,y,n,y


In [4]:
from sklearn.model_selection import train_test_split
x = df['TEXT']
y = df['cEXT']
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size = 0.7,random_state = 42)

In [6]:
import tensorflow_text as text

In [7]:
import tensorflow_hub as hub

In [17]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")


In [10]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

NameError: name 'bert_encoder' is not defined

In [48]:
l = tf.keras.layers.Dropout(0.08, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [49]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [50]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_4 (KerasLayer)     {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [51]:

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [52]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
54/54 [==============================] - 248s 5s/step - loss: 0.6986 - accuracy: 0.5252 - precision: 0.5322 - recall: 0.5540
Epoch 2/10
54/54 [==============================] - 241s 4s/step - loss: 0.7019 - accuracy: 0.5223 - precision: 0.5293 - recall: 0.5552
Epoch 3/10
54/54 [==============================] - 245s 5s/step - loss: 0.7029 - accuracy: 0.5038 - precision: 0.5109 - recall: 0.5870
Epoch 4/10
54/54 [==============================] - 252s 5s/step - loss: 0.6954 - accuracy: 0.5333 - precision: 0.5412 - recall: 0.5449
Epoch 5/10
54/54 [==============================] - 232s 4s/step - loss: 0.6978 - accuracy: 0.5304 - precision: 0.5359 - recall: 0.5768
Epoch 6/10
54/54 [==============================] - 213s 4s/step - loss: 0.6959 - accuracy: 0.5194 - precision: 0.5261 - recall: 0.5620
Epoch 7/10
54/54 [==============================] - 212s 4s/step - loss: 0.6873 - accuracy: 0.5437 - precision: 0.5477 - recall: 0.5939
Epoch 8/10
54/54 [==============================

In [53]:
pred = model.predict(X_test)
pred = np.round(pred).astype(int)
from sklearn import metrics
acc_score = metrics.accuracy_score(y_test, pred)
precision = metrics.precision_score(y_test, pred,zero_division=1)
recall = metrics.recall_score(y_test, pred,zero_division=1)
f1_score = metrics.f1_score(y_test, pred, average='macro',zero_division=1)
print(acc_score)
print(precision)
print(recall)
print(f1_score)

24/24 [==============================] - 102s 4s/step
0.4669365721997301
0.5185185185185185
0.10552763819095477
0.39077340005037087
